In [1]:
#Eyezy-200
import pandas as pd
import xgboost as xgb
import scapy.all as scapy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pickle
from concurrent.futures import ThreadPoolExecutor

# Global variables
initial_bandwidth = 0
initial_packet_rate = 0
initial_total_packets = 0
initial_total_time = 0
label = ""
source_ip = ""
destination_ip = ""
def calculate_total_packets_and_time(packets):
    total_packets = len(packets)
    total_time = packets[-1].time - packets[0].time
    return total_packets, total_time

def calculate_bandwidth_and_packet_rate(packets, total_length, total_time):
    bandwidth = total_length / total_time
    packet_rate = len(packets) / total_time
    return round(bandwidth, 2), round(packet_rate, 2)

def calculate_initial_features(pcap_file):
    packets = scapy.rdpcap(pcap_file)
    total_length = sum(len(packet) for packet in packets)

    with ThreadPoolExecutor() as executor:
        total_packets_and_time_future = executor.submit(calculate_total_packets_and_time, packets)
        total_packets, total_time = total_packets_and_time_future.result()

        bandwidth_and_packet_rate_future = executor.submit(calculate_bandwidth_and_packet_rate, packets, total_length, total_time)
        bandwidth, packet_rate = bandwidth_and_packet_rate_future.result()

    return packets, round(bandwidth, 2), round(packet_rate, 2), round(total_packets, 2), round(total_time, 2)

def load_pretrained_model():
    with open('Original-LightGBM-Dataset-A-80-20.pkl', 'rb') as model_file:
        model = pickle.load(model_file)
    with open('Label-Encoder-Original-LightGBM-Dataset-A-80-20.pkl', 'rb') as encoder_file:
        label_encoder = pickle.load(encoder_file)
    return model, label_encoder

def predict_with_model(model, label_encoder, features):
    features_df = pd.DataFrame([features], columns=[
        'Protocol', 'Source Port', 'Destination Port', 'Total Fwd Pkt', 'Total Bwd Pkt', 'Total Fwd Len', 'Total Bwd Len', 
        'Min Fwd Len', 'Max Fwd Len', 'Mean Fwd Len', 'Std Fwd Len', 'Min Bwd Len', 'Max Bwd Len', 
        'Mean Bwd Len', 'Std Bwd Len', 'Flow Bytes/s', 'Flow Pkt/s', 'Flow Duration', 
        'Bandwidth', 'Packet Rate'
    ])
    features_df = features_df.astype('float64')
    prediction = model.predict(features_df)
    predicted_label = label_encoder.inverse_transform(prediction)[0]
    return predicted_label

def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

    labels = sorted(set(y_true) | set(y_pred))
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    tn, fp, fn, tp = cm.ravel() if cm.size == 4 else (0, 0, 0, 0)
    return accuracy, precision, recall, f1, tp, tn, fp, fn

def group_packets_by_flow(packets):
    flow_dict = {}
    
    for packet in packets:
        if scapy.IP in packet and scapy.TCP in packet:
            src_ip = packet[scapy.IP].src
            dst_ip = packet[scapy.IP].dst
            src_port = packet[scapy.TCP].sport
            dst_port = packet[scapy.TCP].dport
            proto = packet[scapy.IP].proto

            flow_key = (src_ip, dst_ip, src_port, dst_port, proto)
            
            if flow_key not in flow_dict:
                flow_dict[flow_key] = []
            flow_dict[flow_key].append(packet)
    
    return flow_dict

def calculate_features_for_flow(flow, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time):
    features = {}
    packets = flow

    if packets and scapy.IP in packets[0] and scapy.TCP in packets[0]:
        src_ip = packets[0][scapy.IP].src
        dst_ip = packets[0][scapy.IP].dst
        packet_len = len(packets[0])

    # Calculate other features if not a special packet
    total_len = sum(len(pkt) for pkt in packets)
    min_len = min(len(pkt) for pkt in packets)
    max_len = max(len(pkt) for pkt in packets)
    mean_len = total_len / len(packets)
    std_len = pd.Series([len(pkt) for pkt in packets]).std()
    
    first_time = packets[0].time
    last_time = packets[-1].time
    flow_duration = last_time - first_time

    # Ensure flow_duration is not zero to avoid DivisionByZero error
    if flow_duration == 0:
        flow_duration = 0.000012

    flow_bytes_s = total_len / flow_duration
    flow_pkt_s = len(packets) / flow_duration

    features = {
        'Protocol': packets[0][scapy.IP].proto,
        'Source Port': packets[0][scapy.TCP].sport,
        'Destination Port': packets[0][scapy.TCP].dport,
        'Total Fwd Pkt': len(packets) if src_ip == source_ip else 0,
        'Total Fwd Len': total_len if src_ip == source_ip else 0,
        'Min Fwd Len': min_len if src_ip == source_ip else 0,
        'Max Fwd Len': max_len if src_ip == source_ip else 0,
        'Mean Fwd Len': mean_len if src_ip == source_ip else 0,
        'Std Fwd Len': std_len if src_ip == source_ip else 0,
        'Total Bwd Pkt': len(packets) if src_ip != source_ip else 0,
        'Total Bwd Len': total_len if src_ip != source_ip else 0,
        'Min Bwd Len': min_len if src_ip != source_ip else 0,
        'Max Bwd Len': max_len if src_ip != source_ip else 0,
        'Mean Bwd Len': mean_len if src_ip != source_ip else 0,
        'Std Bwd Len': std_len if src_ip != source_ip else 0, 
        'Flow Bytes/s': flow_bytes_s,
        'Flow Pkt/s': flow_pkt_s,
        'Flow Duration': flow_duration,
        'Bandwidth': initial_bandwidth,
        'Packet Rate': initial_packet_rate,
       
    }

    return features, None
def is_request(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_reply(packet):
    return packet.haslayer(scapy.IP) and packet.haslayer(scapy.TCP)

def is_spyware_rule_1(packets):
    normal_traffic_set = set()
    spyware_traffic_set = set()
    
    for packet in packets:
        global destination_ip
        if is_request(packet):
            normal_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            normal_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
        destination_ip = packet[scapy.IP].dst
    
    for packet in packets:
        if is_request(packet):
            spyware_traffic_set.add((packet[scapy.IP].src, packet[scapy.IP].dst, 'RQ'))
        elif is_reply(packet):
            spyware_traffic_set.add((packet[scapy.IP].dst, packet[scapy.IP].src, 'RP'))
    for flow in spyware_traffic_set:
        if flow not in normal_traffic_set:
            return True
    
    return False

def is_spyware_rule_2():
    global destination_ip
    unique_dest_ips = len(set(destination_ip))
    return unique_dest_ips > 3

def is_spyware_rule_3(total_fwd_len, total_bwd_len):
    return total_fwd_len > total_bwd_len

def main():
    pcap_file = 'Eyezy-1.pcap'
    csv_files = ['1-All-Together-Update-Single-26-8-24.csv', '2-All-Together-Update-Single-26-8-24.csv',
                 '3-All-Together-Update-Single-26-8-24.csv','4-All-Together-Update-Single-26-8-24.csv',
                 '5-All-Together-Update-Single-26-8-24.csv']
    
    model, label_encoder = load_pretrained_model()
    
    previous_samples = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)
    y_true = previous_samples['Label']
    y_true_encoded = label_encoder.transform(y_true)

    packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time = calculate_initial_features(pcap_file)
    
    packet_count = 0
    predictions = []
    true_labels = []
    captured_packets = []

    sniff_packets = scapy.sniff(offline=pcap_file)
    for packet in sniff_packets:
        captured_packets.append(packet)
        packet_count += 1

        if packet_count % 200 == 0:
            flow_dict = group_packets_by_flow(captured_packets)
            
            for flow_key, flow_packets in flow_dict.items():
                features, special_label = calculate_features_for_flow(flow_packets, initial_bandwidth, initial_packet_rate, initial_total_packets, initial_total_time)
                
                # Print the global variable values for each flow
                #print(f"Flow Key: {flow_key}")
                #print(f"Bandwidth: {features['Bandwidth']}, Packet Rate: {features['Packet Rate']}, Total-Pkt: {features['Total-Pkt']}, T-Time: {features['T-Time']}")

                if special_label:
                    true_labels.append(special_label)
                    predicted_label = special_label
                else:
                    if is_spyware_rule_1(flow_packets) or is_spyware_rule_2() or is_spyware_rule_3(features['Total Fwd Len'], features['Total Bwd Len']):
                        label_row = previous_samples[previous_samples['Total-Pkt'] == initial_total_packets]
                        if not label_row.empty:
                            label = label_row['Label'].values[0]
                        else:
                            label = 'Unknown'
                    else:
                        label = 'Normal Traffic'
                    true_labels.append(label)
                    predicted_label = predict_with_model(model, label_encoder, features)
                 
                predicted_label_encoded = label_encoder.transform([predicted_label])[0]
                predictions.append(predicted_label_encoded)
                true_labels_encoded = label_encoder.transform(true_labels)
                y_pred = predictions[-len(true_labels_encoded):]
                accuracy, precision, recall, f1, tp, tn, fp, fn = evaluate_model(true_labels_encoded[:len(y_pred)], y_pred)
                          
                print(f"Predicted Label: {predicted_label}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1-Score: {f1}")
               
            
            captured_packets = []
            packet_count = 0

if __name__ == '__main__':
    main()


C:\Users\mimi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


Predicted Label: Eyezy
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0


C:\Users\mimi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


Predicted Label: Eyezy
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0


C:\Users\mimi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


...

F1-Score: 0.8925707302691769
Predicted Label: mLite
Accuracy: 0.8867924528301887
Precision: 0.8935813528301887
Recall: 0.8867924528301887
F1-Score: 0.8901739591542417
Predicted Label: Eyezy
Accuracy: 0.8870967741935484
Precision: 0.8938856741935484
Recall: 0.8870967741935484
F1-Score: 0.8904782849410444
Predicted Label: Eyezy
Accuracy: 0.8873994638069705
Precision: 0.8941883638069705
Recall: 0.8873994638069705
F1-Score: 0.8907809789511932
Predicted Label: Eyezy
Accuracy: 0.8877005347593583
Precision: 0.8944894347593583
Recall: 0.8877005347593583
F1-Score: 0.8910820542738325
Predicted Label: Eyezy
Accuracy: 0.888
Precision: 0.8947889
Recall: 0.888
F1-Score: 0.8913815238584895
Predicted Label: Eyezy
Accuracy: 0.8882978723404256
Precision: 0.8950867723404256
Recall: 0.8882978723404256
F1-Score: 0.891679400516929
Predicted Label: Eyezy
Accuracy: 0.8885941644562334
Precision: 0.8953830644562334
Recall: 0.8885941644562334
F1-Score: 0.8919756969249827
Predicted Label: Air-Droid
Accuracy: 0.88

F1-Score: 0.8970421371214375
Predicted Label: Eyezy
Accuracy: 0.8938775510204081
Precision: 0.9006664510204081
Recall: 0.8938775510204081
F1-Score: 0.8972591595511665
Predicted Label: Eyezy
Accuracy: 0.8940936863543788
Precision: 0.9008825863543788
Recall: 0.8940936863543788
F1-Score: 0.8974752979776519
Predicted Label: Eyezy
Accuracy: 0.8943089430894309
Precision: 0.9010978430894309
Recall: 0.8943089430894309
F1-Score: 0.8976905577911672
Predicted Label: Eyezy
Accuracy: 0.8945233265720081
Precision: 0.901312226572008
Recall: 0.8945233265720081
F1-Score: 0.8979049443382522
Predicted Label: Eyezy
Accuracy: 0.8947368421052632
Precision: 0.9015257421052632
Recall: 0.8947368421052632
F1-Score: 0.8981184629221541
Predicted Label: Eyezy
Accuracy: 0.8949494949494949
Precision: 0.9017383949494949
Recall: 0.8949494949494949
F1-Score: 0.8983311188032651
Predicted Label: Eyezy
Accuracy: 0.8951612903225806
Precision: 0.9019501903225806
Recall: 0.8951612903225806
F1-Score: 0.8985429171995563
Predic

F1-Score: 0.8997632232152398
Predicted Label: Eyezy
Accuracy: 0.896551724137931
Precision: 0.903340624137931
Recall: 0.896551724137931
F1-Score: 0.8999333708268735
Predicted Label: Eyezy
Accuracy: 0.8967213114754098
Precision: 0.9035102114754098
Recall: 0.8967213114754098
F1-Score: 0.9001029605765747
Predicted Label: Eyezy
Accuracy: 0.8968903436988543
Precision: 0.9036792436988543
Recall: 0.8968903436988543
F1-Score: 0.9002719952034415
Predicted Label: Eyezy
Accuracy: 0.8970588235294118
Precision: 0.9038477235294118
Recall: 0.8970588235294118
F1-Score: 0.9004404774286687
Predicted Label: Eyezy
Accuracy: 0.8972267536704731
Precision: 0.9040156536704731
Recall: 0.8972267536704731
F1-Score: 0.9006084099556955
Predicted Label: Eyezy
Accuracy: 0.8973941368078175
Precision: 0.9041830368078175
Recall: 0.8973941368078175
F1-Score: 0.900775795470348
Predicted Label: Eyezy
Accuracy: 0.8975609756097561
Precision: 0.9043498756097561
Recall: 0.8975609756097561
F1-Score: 0.9009426366409848
Predicted

F1-Score: 0.8959435731462059
Predicted Label: TiSpy
Accuracy: 0.8913342503438789
Precision: 0.8981231503438789
Recall: 0.8913342503438789
F1-Score: 0.8947158223722681
Predicted Label: mSpy
Accuracy: 0.8901098901098901
Precision: 0.89689879010989
Recall: 0.8901098901098901
F1-Score: 0.8934914444917209
Predicted Label: mLite
Accuracy: 0.8888888888888888
Precision: 0.8956777888888888
Recall: 0.8888888888888888
F1-Score: 0.8922704256243457
Predicted Label: Eyezy
Accuracy: 0.8890410958904109
Precision: 0.8958299958904109
Recall: 0.8890410958904109
F1-Score: 0.8924226348282557
Predicted Label: Eyezy
Accuracy: 0.8891928864569083
Precision: 0.8959817864569083
Recall: 0.8891928864569083
F1-Score: 0.8925744275903672
Predicted Label: Eyezy
Accuracy: 0.889344262295082
Precision: 0.896133162295082
Recall: 0.889344262295082
F1-Score: 0.8927258056174127
Predicted Label: Eyezy
Accuracy: 0.8894952251023193
Precision: 0.8962841251023193
Recall: 0.8894952251023193
F1-Score: 0.8928767706068101
Predicted L

Recall: 0.8946745562130177
F1-Score: 0.8980561761401361
Predicted Label: Eyezy
Accuracy: 0.8947990543735225
Precision: 0.9015879543735225
Recall: 0.8947990543735225
F1-Score: 0.8981806760790109
Predicted Label: Eyezy
Accuracy: 0.8949232585596222
Precision: 0.9017121585596222
Recall: 0.8949232585596222
F1-Score: 0.8983048820387903
Predicted Label: Eyezy
Accuracy: 0.8950471698113207
Precision: 0.9018360698113207
Recall: 0.8950471698113207
F1-Score: 0.8984287950594966
Predicted Label: Eyezy
Accuracy: 0.8951707891637221
Precision: 0.901959689163722
Recall: 0.8951707891637221
F1-Score: 0.8985524161762525
Predicted Label: Eyezy
Accuracy: 0.8952941176470588
Precision: 0.9020830176470588
Recall: 0.8952941176470588
F1-Score: 0.8986757464193084
Predicted Label: Eyezy
Accuracy: 0.8954171562867215
Precision: 0.9022060562867215
Recall: 0.8954171562867215
F1-Score: 0.8987987868140734
Predicted Label: Eyezy
Accuracy: 0.8955399061032864
Precision: 0.9023288061032864
Recall: 0.8955399061032864
F1-Score